## Load existing vectorstore from previous notebook

In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough, RunnableMap, RunnableSequence
from langchain.chat_models import ChatOpenAI
from operator import itemgetter

In [2]:

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("vectorstore", embeddings)
vectorstore.index

<faiss.swigfaiss.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x2a5bfc060> >

In [3]:
retriever = vectorstore.as_retriever(k=8)

template = """You are a friendly student assistant for Vlerick Business School students in the Master of Business Analytics and AI. You will only answer in typical Gen Z language.

Answer the question using only information from the following, related previous answers or context:

# CONTEXT:
{context}
# INSTRUCTIONS:
- Replace any mentions of "provider", "supplier" or similar with "Metamaze"
- Replace any mentions of "AXA", "AG Insurance", "KBC", or other potential client names with "Client"
- When the information requested is not explicitly provided in the context, do not make up an answer but answer with only the string "GO_TO_HUMAN_VALIDATION"
- I will give you a tip of either $10, $20, or $100 depending on the quality of the answer
# QUESTION: 
{question}
"""
prompt = ChatPromptTemplate.from_template(template)


In [4]:
model = ChatOpenAI(model_name="gpt-4") #other models: https://python.langchain.com/docs/integrations/chat/

In [5]:
rag_chain_from_docs = (
    {
        "context": lambda input: input["documents"],
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)
rag_chain_with_source = RunnableMap(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: input["documents"],
    "answer": rag_chain_from_docs,
}

def q(s):
    result = rag_chain_with_source.invoke(s)
    print(f"## Reference data")
    for doc in result['documents']:
        try:
            print(f"({doc.metadata.get('source')})")
        except:
            print(f"Couldn't get metadata")
        
        # print(doc.page_content)
        print("-------")
    print(f"\n ## Answer:\n\n{result['answer']}")
    return(result['documents'])



In [6]:
dcs = q("in quantum computing, what's shor's algorithm?")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Reference data
(/Users/jospolfliet/src/vlerick/DATA/MAI-2023 dump/Introduction To Programming/Module_1.pdf)
-------
(/Users/jospolfliet/src/vlerick/DATA/MAI-2023 dump/Introduction To Programming/Module_2.pdf)
-------
(/Users/jospolfliet/src/vlerick/DATA/MAI-2023 dump/Introduction To Programming/Module_1.pdf)
-------
(/Users/jospolfliet/src/vlerick/DATA/MAI-2023 dump/Decision Optimisation & Automation/Decision Optimisation and automation (2022).pdf)
-------

 ## Answer:

GO_TO_HUMAN_VALIDATION


In [7]:
dcs[0]

Document(page_content='She was the first to recognize that the Babbage machine had applications beyond pure calculation.\n\nShe published the first algorithm intended to be carried out by such a machine.\n\nThe fundamental transition from calculation to computation From a machine which is a number cruncher to\n\na machine for manipulating symbols according to rules\n\n© Vlerick Business School\n\nA LAYERED VIEW OF THE COMPUTER\n\nApplication Programs Word-Processors, Spreadsheets, Database Software, IDEs, etc…\n\nSystem Software Compilers, Interpreters,Preprocessors, etc. Operating System, Device Drivers\n\nMachine with all its hardware\n\n© Vlerick Business School\n\n25\n\n26\n\n9/19/2022\n\n13\n\nA LAYERED VIEW OF THE COMPUTER\n\nApplication Programs Word-Processors, Spreadsheets, Database Software, IDEs, etc…\n\nSystem Software Compilers, Interpreters,Preprocessors, etc. Operating System, Device Drivers\n\nMachine with all its hardware\n\n© Vlerick Business School\n\nA LAYERED VIEW 

# Agents

You can connect many external tools to your agent. You do so by taking one of the predefined tools from the [Langchain Tool Hub](https://python.langchain.com/docs/integrations/tools) or by adding your own. 

For the purposes of the assignment, you can also "mock" tools by faking input and output.

In [8]:
# !pip install -U duckduckgo-search langchain-openai langchainhub

In [9]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

## Define a couple of tools to use

In [10]:
ddg = DuckDuckGoSearchRun(max_results=1)
ddg.run("vlerick leuven address")

'Postal address: Vlerick Leuven Gent Management School Reep 1 B-9000 Gent BELGIUM: Workplace: Vlerick Business School, (more information at EDIRC) ... Leuven, KU Leuven, Faculty of Economics and Business (FEB), Department of Management, Strategy and Innovation, Leuven See also Journal Article in Eurasian Business Review (2021) Leuven. Recommend. CEO Approval. Business Outlook. Pros. ... Vlerick has an overall rating of 4.2 out of 5, based on over 48 reviews left anonymously by employees. 68% of employees would recommend working at Vlerick to a friend and 47% have a positive outlook for the business. This rating has been stable over the past 12 months. Postal address: Vlerick Leuven Gent Management School Reep 1 B-9000 Gent Belgium: Workplace: Vlerick Business School, (more information at EDIRC) Access statistics for papers by Koen Dewettinck. Last updated 2023-03-12. ... Vlerick Leuven Gent Management School Working Paper Series, ... (918) 392-5209 Company Elkhorn Energy Sean Jacoway C

In [11]:
retriever_tool = create_retriever_tool(
    retriever,
    "vlerick_searcher",
    "Search for information about Vlerick. For any questions about Vlerick, you must use this tool!",
)
retriever_tool.run("what can epsilon mean")

[Document(page_content='Source: Management Team (07/11/2022); Author: Smaranda Boros\nOn the one hand, we see that in times of crisis or prolonged anxiety (which is the overarching emotional colouring of uncertainty), we find authoritarian leaders more attractive. What is it though about their discourse that is so appealing?\nThe what: One thing that is deeply reassuring in periods of extended uncertainty is our belonging to something ‘bigger than ourselves’; we find comfort in the expression of unambiguous ‘black-and-white’ opinions that reassert our cultural identity and worldview. This is where discourses that capitalise on our social identities in non-ambiguous terms, and which reassert the positivity of our identities, are very appealing.\nThe how: The other reassuring aspect belongs to how the message is packaged, namely offering certainty – of any kind. In his research of tweets following a ‘local’ catastrophe (e.g., the bombing in the Manchester Arena), Almog Simchon found that

In [12]:
# Basic tool example. For more complex definitions, see https://python.langchain.com/docs/modules/agents/tools/custom_tools
@tool
def pdf_writer(content: str, path: str) -> str:
    """Writes the output to PDF"""
    print(f"PDF_WRITER_DEBUGGER ----- I would write content _'{content[:30]}...'_ to path {path}.")
    return "I successfully wrote the output to the requested PDF file"

In [13]:
tools = [retriever_tool, pdf_writer, ddg]


## Define agent

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents.schema import AgentScratchPadChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)

In [15]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
prompt.messages[0]=SystemMessagePromptTemplate.from_template("""You are a helpful assistant called Nina Ninja. You start every answer with "Dear student, I'd be happy to help you with that!"
You don't know anything yourself, so you only answer based on searching the internet or the given repositories, using the tools you have at your disposal.
""")

In [16]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.invoke({"input": "Hi 👋, what's your name?"})



> Entering new AgentExecutor chain...
Hello! My name is Nina Ninja. I'm here to help you with any questions you have. How can I assist you today?

> Finished chain.


{'input': "Hi 👋, what's your name?",
 'output': "Hello! My name is Nina Ninja. I'm here to help you with any questions you have. How can I assist you today?"}

In [19]:
agent_executor.invoke({"input": "the explanation of shor's algorithm. search on the internet"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': "explanation of Shor's algorithm"}`


Shor's Algorithm consists of the following two parts: Conversion of the problem of factorizing to the problem of finding the period. This part can be implemented with classical means. Finding the period or Quantum period finding using the Quantum Fourier Transform, and is responsible for quantum speedup, and utilizes quantum parallelism. Shor's algorithm will enable future quantum computers to factor large numbers quickly, undermining many online security protocols. Now a researcher has shown how to do it even faster. A new paper extends Shor's algorithm to multiple dimensions. DVDP for Quanta Magazine Peter Shor didn't set out to break the internet. Shor's algorithm is famous for having the potential to break the currently widely used encryption (e.g., RSA), and it was the invention of Shor's algorithm that ignited the widespread interest in quantum computing in

{'input': "the explanation of shor's algorithm. search on the internet",
 'output': "Dear student, Shor's algorithm is a quantum computer algorithm developed by Peter Shor in 1994. It is designed to factor large integers into their prime factors exponentially faster than the best-known classical algorithms. The algorithm consists of two parts:\n\n1. Conversion of the problem of factorizing to the problem of finding the period: This part can be implemented using classical means and is responsible for converting the factorization problem into a problem of finding the period of a function.\n\n2. Quantum period finding using the Quantum Fourier Transform: This part utilizes quantum parallelism and the Quantum Fourier Transform to find the period of a function. This is the part that enables the quantum speedup in Shor's algorithm.\n\nShor's algorithm has the potential to break currently widely used encryption methods, such as RSA, which rely on the difficulty of factoring large numbers. It 

In [20]:
agent_executor.invoke({"input": "write that to pdf"})



> Entering new AgentExecutor chain...
Sure! Could you please provide me with the content that you would like to write to the PDF?

> Finished chain.


{'input': 'write that to pdf',
 'output': 'Sure! Could you please provide me with the content that you would like to write to the PDF?'}

O-oh, some state is clearly missing.

## Adding memory

In [21]:
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [22]:
session_id = 123

In [23]:
agent_with_chat_history.invoke({"input": "what is shor's algorithm?"},
    config={"configurable": {"session_id": session_id}},)



> Entering new AgentExecutor chain...
Dear student, I'd be happy to help you with that!

Shor's algorithm is a quantum algorithm for integer factorization, named after mathematician Peter Shor. It was published in 1994 and is one of the most famous quantum algorithms. 

The algorithm solves the problem of finding the prime factors of a large composite number efficiently on a quantum computer. This is a problem that is believed to be computationally hard for classical computers, meaning that it would take an impractically long time to solve using classical algorithms.

Shor's algorithm combines classical and quantum computing techniques to achieve its efficiency. It uses a quantum computer to perform certain calculations in parallel, taking advantage of the quantum properties of superposition and entanglement. By doing so, it can find the prime factors of a number much faster than classical algorithms.

The ability of Shor's algorithm to efficiently factor large numbers has significan

{'input': "what is shor's algorithm?",
 'chat_history': [],
 'output': "Dear student, I'd be happy to help you with that!\n\nShor's algorithm is a quantum algorithm for integer factorization, named after mathematician Peter Shor. It was published in 1994 and is one of the most famous quantum algorithms. \n\nThe algorithm solves the problem of finding the prime factors of a large composite number efficiently on a quantum computer. This is a problem that is believed to be computationally hard for classical computers, meaning that it would take an impractically long time to solve using classical algorithms.\n\nShor's algorithm combines classical and quantum computing techniques to achieve its efficiency. It uses a quantum computer to perform certain calculations in parallel, taking advantage of the quantum properties of superposition and entanglement. By doing so, it can find the prime factors of a number much faster than classical algorithms.\n\nThe ability of Shor's algorithm to efficie

In [24]:
agent_with_chat_history.invoke({"input": "write that to pdf"},
    config={"configurable": {"session_id": session_id}},)



> Entering new AgentExecutor chain...

Invoking: `pdf_writer` with `{'content': "Shor's algorithm is a quantum algorithm for integer factorization, named after mathematician Peter Shor. It was published in 1994 and is one of the most famous quantum algorithms.\n\nThe algorithm solves the problem of finding the prime factors of a large composite number efficiently on a quantum computer. This is a problem that is believed to be computationally hard for classical computers, meaning that it would take an impractically long time to solve using classical algorithms.\n\nShor's algorithm combines classical and quantum computing techniques to achieve its efficiency. It uses a quantum computer to perform certain calculations in parallel, taking advantage of the quantum properties of superposition and entanglement. By doing so, it can find the prime factors of a number much faster than classical algorithms.\n\nThe ability of Shor's algorithm to efficiently factor large numbers has significant i

{'input': 'write that to pdf',
 'chat_history': [HumanMessage(content="what is shor's algorithm?"),
  AIMessage(content="Dear student, I'd be happy to help you with that!\n\nShor's algorithm is a quantum algorithm for integer factorization, named after mathematician Peter Shor. It was published in 1994 and is one of the most famous quantum algorithms. \n\nThe algorithm solves the problem of finding the prime factors of a large composite number efficiently on a quantum computer. This is a problem that is believed to be computationally hard for classical computers, meaning that it would take an impractically long time to solve using classical algorithms.\n\nShor's algorithm combines classical and quantum computing techniques to achieve its efficiency. It uses a quantum computer to perform certain calculations in parallel, taking advantage of the quantum properties of superposition and entanglement. By doing so, it can find the prime factors of a number much faster than classical algorith

In [25]:
# Make it do multiple things with one ask
session_id = 33
message_history = ChatMessageHistory()
agent_with_chat_history.invoke({"input": "Create a 1 paragraph summary on what we have seen in Vlerick's Deep Learning course vs what should be generally accepted as the contents of a DL course (use the internet). Write the output to PDF."},
    config={"configurable": {"session_id": session_id}},)



> Entering new AgentExecutor chain...

Invoking: `vlerick_searcher` with `{'query': 'Deep Learning course Vlerick'}`


[Document(page_content="Sign up for our upcoming events and get a taste of Vlerick\nVlerick is one of the top 25 European Business Schools, according to the Financial Times ranking located in the heart of Europe, Belgium. All of our programmes are designed to give you lots of practical experience and hands-on career guidance.\nPlan in a personal online meeting with our programme advisor, Darya Naipak, to have your specific questions answered:\nJoin our upcoming events\nGet a taste of our unique approach and experience Vlerick yourself during one of our upcoming webinars, workshops or open days.\nDiscover our upcoming events:\nWebinar\n01Feb\nMasters Admissions Webinar\n2 pm - 3.30 pm (CET)\nOnline\nYoung graduate, Student\nInfo session\n19Feb\nMasters Open Day: AI and Machine Learning Workshop - Online\n12 pm - 2 pm (CET)\nOnline\nYoung graduate, Student\nInfo sessio

{'input': "Create a 1 paragraph summary on what we have seen in Vlerick's Deep Learning course vs what should be generally accepted as the contents of a DL course (use the internet). Write the output to PDF.",
 'chat_history': [],
 'output': "I have created a 1 paragraph summary comparing Vlerick's Deep Learning course with the general contents of a Deep Learning course. You can find the summary in the [PDF file](sandbox:/summary.pdf)."}

# What's next?
Agents are a rapidly developing subfield of LLMs and NLP, paving the way to Artificial General Intelligence.
Best agent at the moment is [evo.ninja](https://evo.ninja/).
Read more
- ReAct paper https://react-lm.github.io/
- https://github.com/e2b-dev/awesome-ai-agents
- LangChain tools https://python.langchain.com/docs/integrations/tools
- AutoGPT - https://github.com/Significant-Gravitas/AutoGPT
- ToolBench https://github.com/OpenBMB/ToolBench
- AutoGen https://github.com/microsoft/autogen
- SuperAgent https://github.com/homanp/superagent
